In [1]:
import pandas as pd

covid_states_df = 'C:\Bootcamp-project\Project-Three\COVID-19_Data-master\Data\COVID_AU_state.csv'
df = pd.read_csv(covid_states_df)
print(df.head(5))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Bootcamp-project\\Project-Three\\COVID-19_Data-master\\Data\\COVID_AU_state.csv'

In [3]:
#Read covid data from url
url = 'https://raw.githubusercontent.com/M3IT/COVID-19_Data/master/Data/COVID_AU_state.csv'
covid_df = pd.read_csv(url)
covid_df

,date,state,state_abbrev,confirmed,confirmed_cum,deaths,deaths_cum,tests,tests_cum,positives,...,recovered,recovered_cum,hosp,hosp_cum,icu,icu_cum,vent,vent_cum,vaccines,vaccines_cum
0,2020-01-25,Australian Capital Territory,ACT,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0,0,0,0,0
1,2020-01-25,New South Wales,NSW,3,3,0,0,0,0.0,3,...,0,0.0,0,0,0,0,0,0,0,0
2,2020-01-25,Northern Territory,NT,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0,0,0,0,0
3,2020-01-25,Queensland,QLD,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0,0,0,0,0
4,2020-01-25,South Australia,SA,0,0,0,0,0,0.0,0,...,0,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8115,2022-11-04,Queensland,QLD,4067,1668438,14,2326,0,NaN,4067,...,0,NaN,0,105,1,3,0,1,0,0
8116,2022-11-04,South Australia,SA,3018,780614,29,1067,15973,6125312.0,3018,...,0,NaN,9,53,1,6,1,1,0,0
8117,2022-11-04,Tasmania,TAS,1320,253710,5,199,0,NaN,1320,...,0,NaN,-3,3,0,1,0,0,0,0
8118,2022-11-04,Victoria,VIC,10153,2648933,24,5847,5257,22530588.0,10153,...,0,NaN,59,231,1,7,0,2,0,0


In [17]:
#Read Flu data from excel file
#Read first worksheet
flu_df1 = pd.read_excel('Data/national-notifiable-diseases-surveillance-system-nndss-public-dataset-influenza-laboratory-confirmed-dataset.xlsx', sheet_name=0, skiprows=[0, 1, 2, 3])
flu_df1

,Week Ending (Friday),State,Age group,Sex,Indigenous status,Type/Subtype
0,2016-01-01,NSW,00-04,Female,Not available,A(H1N1)pdm09
1,2016-01-01,NSW,00-04,Female,Not available,A(H1N1)pdm09
2,2016-01-01,NSW,00-04,Female,Not available,A(H1N1)pdm09
3,2016-01-01,NSW,00-04,Female,Not available,A(unsubtyped)
4,2016-01-01,NSW,00-04,Male,Not available,A(H1N1)pdm09
...,...,...,...,...,...,...
344414,2008-01-04,WA,65-69,Male,Non-Indigenous,B
344415,2008-01-04,WA,75-79,Male,Unknown,A(unsubtyped)
344416,2008-01-04,WA,80-84,Male,Non-Indigenous,A(unsubtyped)
344417,2008-01-04,WA,80-84,Male,Non-Indigenous,A(unsubtyped)


In [19]:
#Read second worksheet
flu_df2 = pd.read_excel('Data/national-notifiable-diseases-surveillance-system-nndss-public-dataset-influenza-laboratory-confirmed-dataset.xlsx', sheet_name=1, skiprows=[0, 1, 2, 3])
flu_df2

,Week Ending (Friday),State,Age group,Sex,Indigenous status,Type/Subtype
0,2019-01-04,NSW,00-04,Female,Not available,A(H1N1)pdm09
1,2019-01-04,NSW,00-04,Female,Not available,A(unsubtyped)
2,2019-01-04,NSW,00-04,Female,Not available,A(unsubtyped)
3,2019-01-04,NSW,00-04,Female,Not available,A(unsubtyped)
4,2019-01-04,NSW,00-04,Female,Not available,A(unsubtyped)
...,...,...,...,...,...,...
395830,2016-01-01,WA,45-49,Female,Non-Indigenous,A(H1N1)pdm09
395831,2016-01-01,WA,65-69,Female,Non-Indigenous,B
395832,2016-01-01,WA,75-79,Male,Non-Indigenous,A(H3N2)
395833,2016-01-01,WA,65-69,Female,Non-Indigenous,B


In [20]:
#Read third worksheet
flu_df3 = pd.read_excel('Data/national-notifiable-diseases-surveillance-system-nndss-public-dataset-influenza-laboratory-confirmed-dataset.xlsx', sheet_name=2, skiprows=[0, 1, 2, 3])
flu_df3

,Week Ending (Friday),State,Age group,Sex,Indigenous status,Type/Subtype
0,2021-12-31,NSW,00-04,Female,Not available,A(unsubtyped)
1,2021-12-31,NSW,00-04,Female,Not available,A(unsubtyped)
2,2021-12-31,NSW,05-09,Female,Not available,A(unsubtyped)
3,2021-12-31,NSW,05-09,Male,Not available,A(unsubtyped)
4,2021-12-31,NSW,05-09,Male,Not available,A(unsubtyped)
...,...,...,...,...,...,...
331289,2019-01-04,WA,75-79,Male,Non-Indigenous,A(unsubtyped)
331290,2019-01-04,WA,80-84,Male,Non-Indigenous,B
331291,2019-01-04,WA,85+,Female,Non-Indigenous,A(unsubtyped)
331292,2019-01-04,WA,80-84,Male,Non-Indigenous,B


In [30]:
#Concatenate the dataframes
flu_df = pd.concat([flu_df1, flu_df2, flu_df3])
flu_df.shape

(1071548, 6)

In [29]:
import datetime as dt
#convert the week ending date to datetime
flu_df['Week Ending (Friday)'] = pd.to_datetime(flu_df['Week Ending (Friday)'])
#Group by year and week ending date
flu_grouped = flu_df.groupby([flu_df['Week Ending (Friday)'].dt.isocalendar().year, flu_df['Week Ending (Friday)'].dt.isocalendar().week])
flu_grouped.head()

,Week Ending (Friday),State,Age group,Sex,Indigenous status,Type/Subtype
0,2016-01-01,NSW,00-04,Female,Not available,A(H1N1)pdm09
1,2016-01-01,NSW,00-04,Female,Not available,A(H1N1)pdm09
2,2016-01-01,NSW,00-04,Female,Not available,A(H1N1)pdm09
3,2016-01-01,NSW,00-04,Female,Not available,A(unsubtyped)
4,2016-01-01,NSW,00-04,Male,Not available,A(H1N1)pdm09
...,...,...,...,...,...,...
328631,2019-01-11,NSW,00-04,Female,Not available,A(H1N1)pdm09
328632,2019-01-11,NSW,00-04,Female,Not available,A(H1N1)pdm09
328633,2019-01-11,NSW,00-04,Female,Not available,A(H1N1)pdm09
328634,2019-01-11,NSW,00-04,Female,Not available,A(H1N1)pdm09


In [23]:
#plot the data
flu_df.plot.line(x='Week Ending(Friday)', y='Number of cases')

KeyError: 'Year'